# 第五節 – 多代理協調器

展示使用 Foundry Local 的簡單雙代理流程（研究員 -> 編輯）。


### 解釋：依賴安裝
安裝 `foundry-local-sdk` 和 `openai`，用於本地模型訪問和聊天完成。具備冪等性。


# 情境
實現一個簡單的雙代理協作模式：
- **研究員代理**收集簡潔的事實要點
- **編輯代理**重寫內容以達到高層次的清晰度

展示每個代理共享記憶、按順序傳遞中間輸出，以及一個簡單的管道功能。可擴展至更多角色（例如評論員、驗證員）或平行分支。

**環境變數：**
- `FOUNDRY_LOCAL_ALIAS` - 預設使用的模型（預設值：phi-4-mini）
- `AGENT_MODEL_PRIMARY` - 主要代理模型（覆蓋 ALIAS）
- `AGENT_MODEL_EDITOR` - 編輯代理模型（預設為主要模型）

**SDK 參考：** https://github.com/microsoft/Foundry-Local/tree/main/sdk/python/foundry_local

**運作方式：**
1. **FoundryLocalManager** 自動啟動 Foundry Local 服務
2. 下載並載入指定的模型（或使用快取版本）
3. 提供一個與 OpenAI 相容的端點以進行互動
4. 每個代理可使用不同的模型以完成專業化任務
5. 內建重試邏輯可優雅地處理暫時性故障

**主要功能：**
- ✅ 自動服務發現與初始化
- ✅ 模型生命週期管理（下載、快取、載入）
- ✅ 與 OpenAI SDK 相容，提供熟悉的 API
- ✅ 支援多模型以實現代理專業化
- ✅ 強大的錯誤處理與重試邏輯
- ✅ 本地推理（不需要雲端 API）


In [16]:
# Install dependencies
!pip install -q foundry-local-sdk openai

### 解釋：核心導入與類型提示
引入資料類別以存儲代理訊息，並使用類型提示提高清晰度。導入 Foundry Local 管理器和 OpenAI 客戶端以支援後續代理操作。


In [17]:
from dataclasses import dataclass, field
from typing import List
import os
from foundry_local import FoundryLocalManager
from openai import OpenAI

### 解釋：模型初始化（SDK 模式）
使用 Foundry Local Python SDK 進行穩健的模型管理：
- **FoundryLocalManager(alias)** - 自動啟動服務並通過別名加載模型
- **get_model_info(alias)** - 將別名解析為具體的模型 ID
- **manager.endpoint** - 提供 OpenAI 客戶端的服務端點
- **manager.api_key** - 提供 API 密鑰（本地使用時可選）
- 支援為不同代理（主要代理與編輯代理）使用獨立模型
- 內建重試邏輯，採用指數回退以提高韌性
- 連接驗證以確保服務已準備就緒

**主要 SDK 模式：**
```python
manager = FoundryLocalManager(alias)
model_info = manager.get_model_info(alias)
client = OpenAI(base_url=manager.endpoint, api_key=manager.api_key)
```

**生命週期管理：**
- 管理器全局存儲以便正確清理
- 每個代理可使用不同模型以實現專業化
- 自動服務發現與連接處理
- 在失敗時進行指數回退的優雅重試

這確保在代理編排開始之前完成正確的初始化。

**參考資料：** https://github.com/microsoft/Foundry-Local/tree/main/sdk/python/foundry_local


In [18]:
import time

# Environment configuration
PRIMARY_ALIAS = os.getenv('AGENT_MODEL_PRIMARY', os.getenv('FOUNDRY_LOCAL_ALIAS', 'phi-4-mini'))
EDITOR_ALIAS = os.getenv('AGENT_MODEL_EDITOR', PRIMARY_ALIAS)

# Store managers globally for proper lifecycle management
primary_manager = None
editor_manager = None

def init_model(alias: str, max_retries: int = 3):
    """Initialize Foundry Local manager with retry logic.
    
    Args:
        alias: Model alias to initialize
        max_retries: Number of retry attempts with exponential backoff
    
    Returns:
        Tuple of (manager, client, model_id, endpoint)
    """
    delay = 2.0
    last_err = None
    
    for attempt in range(1, max_retries + 1):
        try:
            print(f"[Init] Starting Foundry Local for '{alias}' (attempt {attempt}/{max_retries})...")
            
            # Initialize manager - this starts the service and loads the model
            manager = FoundryLocalManager(alias)
            
            # Get model info to retrieve the actual model ID
            model_info = manager.get_model_info(alias)
            model_id = model_info.id
            
            # Create OpenAI client with manager's endpoint
            client = OpenAI(
                base_url=manager.endpoint,
                api_key=manager.api_key or 'not-needed'
            )
            
            # Verify the connection with a simple test
            models = client.models.list()
            print(f"[OK] Initialized '{alias}' -> {model_id} at {manager.endpoint}")
            
            return manager, client, model_id, manager.endpoint
            
        except Exception as e:
            last_err = e
            if attempt < max_retries:
                print(f"[Retry {attempt}/{max_retries}] Failed to init '{alias}': {e}")
                print(f"[Retry] Waiting {delay:.1f}s before retry...")
                time.sleep(delay)
                delay *= 2
            else:
                print(f"[ERROR] Failed to initialize '{alias}' after {max_retries} attempts")
    
    raise RuntimeError(f"Failed to initialize '{alias}' after {max_retries} attempts: {last_err}")

# Initialize primary model (for researcher)
print(f"\n{'='*80}")
print(f"Initializing Primary Model: {PRIMARY_ALIAS}")
print('='*80)
primary_manager, primary_client, PRIMARY_MODEL_ID, primary_endpoint = init_model(PRIMARY_ALIAS)

# Initialize editor model (may be same as primary)
if EDITOR_ALIAS != PRIMARY_ALIAS:
    print(f"\n{'='*80}")
    print(f"Initializing Editor Model: {EDITOR_ALIAS}")
    print('='*80)
    editor_manager, editor_client, EDITOR_MODEL_ID, editor_endpoint = init_model(EDITOR_ALIAS)
else:
    print(f"\n[Info] Editor using same model as primary")
    editor_manager = primary_manager
    editor_client, EDITOR_MODEL_ID = primary_client, PRIMARY_MODEL_ID
    editor_endpoint = primary_endpoint

print(f"\n{'='*80}")
print(f"[Configuration Summary]")
print('='*80)
print(f"  Primary Agent:")
print(f"    - Alias: {PRIMARY_ALIAS}")
print(f"    - Model: {PRIMARY_MODEL_ID}")
print(f"    - Endpoint: {primary_endpoint}")
print(f"\n  Editor Agent:")
print(f"    - Alias: {EDITOR_ALIAS}")
print(f"    - Model: {EDITOR_MODEL_ID}")
print(f"    - Endpoint: {editor_endpoint}")
print('='*80)



Initializing Primary Model: phi-4-mini
[Init] Starting Foundry Local for 'phi-4-mini' (attempt 1/3)...
[OK] Initialized 'phi-4-mini' -> Phi-4-mini-instruct-cuda-gpu:4 at http://127.0.0.1:59959/v1

Initializing Editor Model: gpt-oss-20b
[Init] Starting Foundry Local for 'gpt-oss-20b' (attempt 1/3)...
[OK] Initialized 'gpt-oss-20b' -> gpt-oss-20b-cuda-gpu:1 at http://127.0.0.1:59959/v1

[Configuration Summary]
  Primary Agent:
    - Alias: phi-4-mini
    - Model: Phi-4-mini-instruct-cuda-gpu:4
    - Endpoint: http://127.0.0.1:59959/v1

  Editor Agent:
    - Alias: gpt-oss-20b
    - Model: gpt-oss-20b-cuda-gpu:1
    - Endpoint: http://127.0.0.1:59959/v1


### 解釋：Agent 和 Memory 類別
定義輕量級的 `AgentMsg` 用於記憶條目，以及封裝的 `Agent`：
- **系統角色** - Agent 的角色和指令
- **訊息歷史** - 維持對話的上下文
- **act() 方法** - 執行操作並正確處理錯誤

Agent 可以使用不同的模型（主要模型與編輯器模型），並為每個 Agent 維持獨立的上下文。此模式能夠實現：
- 操作間的記憶持久性
- 每個 Agent 的靈活模型分配
- 錯誤隔離與恢復
- 簡易的鏈接與編排


In [19]:
@dataclass
class AgentMsg:
    role: str
    content: str

@dataclass
class Agent:
    name: str
    system: str
    client: OpenAI = None  # Allow per-agent client assignment
    model_id: str = None   # Allow per-agent model
    memory: List[AgentMsg] = field(default_factory=list)

    def _history(self):
        """Return chat history in OpenAI messages format including system + memory."""
        msgs = [{'role': 'system', 'content': self.system}]
        for m in self.memory[-6:]:  # Keep last 6 messages to avoid context overflow
            msgs.append({'role': m.role, 'content': m.content})
        return msgs

    def act(self, prompt: str, temperature: float = 0.4, max_tokens: int = 300):
        """Send a prompt, store user + assistant messages in memory, and return assistant text.
        
        Args:
            prompt: User input/task for the agent
            temperature: Sampling temperature (0.0-1.0)
            max_tokens: Maximum tokens to generate
        
        Returns:
            Assistant response text
        """
        # Use agent-specific client/model or fall back to primary
        client_to_use = self.client or primary_client
        model_to_use = self.model_id or PRIMARY_MODEL_ID
        
        self.memory.append(AgentMsg('user', prompt))
        
        try:
            # Build messages including system prompt and history
            messages = self._history() + [{'role': 'user', 'content': prompt}]
            
            resp = client_to_use.chat.completions.create(
                model=model_to_use,
                messages=messages,
                max_tokens=max_tokens,
                temperature=temperature,
            )
            
            # Validate response
            if not resp.choices:
                raise RuntimeError("No completion choices returned")
            
            out = resp.choices[0].message.content or ""
            
            if not out:
                raise RuntimeError("Empty response content")
            
        except Exception as e:
            out = f"[ERROR:{self.name}] {type(e).__name__}: {str(e)}"
            print(f"[Agent Error] {self.name}: {type(e).__name__}: {str(e)}")
        
        self.memory.append(AgentMsg('assistant', out))
        return out

print("[INFO] Agent classes initialized with Foundry SDK support")
print(f"[INFO] Using OpenAI SDK version: {OpenAI.__module__}")


[INFO] Agent classes initialized with Foundry SDK support
[INFO] Using OpenAI SDK version: openai


### 解釋：協調式管道
建立兩個專門的代理：
- **研究員**：使用主要模型，收集事實資訊
- **編輯員**：可使用不同的模型（如果已配置），進行精煉和重寫

`pipeline` 函數：
1. 研究員收集原始資訊
2. 編輯員將其精煉為適合執行的輸出
3. 返回中間結果和最終結果

此模式的優勢包括：
- 模型專業化（不同角色使用不同模型）
- 通過多階段處理提升品質
- 資訊轉換的可追溯性
- 容易擴展至更多代理或平行處理


In [ ]:
# Create specialized agents with optional model assignment
researcher = Agent(
    name='Researcher',
    system='You collect concise factual bullet points.',
    client=primary_client,
    model_id=PRIMARY_MODEL_ID
)

editor = Agent(
    name='Editor',
    system='You rewrite content for clarity and an executive, action-focused tone.',
    client=editor_client,
    model_id=EDITOR_MODEL_ID
)

def pipeline(q: str, verbose: bool = True):
    """Execute multi-agent pipeline: Researcher -> Editor.
    
    Args:
        q: User question/task
        verbose: Print intermediate outputs
    
    Returns:
        Dictionary with research, final outputs, and metadata
    """
    if verbose:
        print(f"[Pipeline] Question: {q}\n")
    
    # Stage 1: Research
    if verbose:
        print("[Stage 1: Research]")
    research = researcher.act(q)
    if verbose:
        print(f"Output: {research[:200]}...\n")
    
    # Stage 2: Editorial refinement
    if verbose:
        print("[Stage 2: Editorial Refinement]")
    rewrite = editor.act(
        f"Rewrite professionally with a 1-sentence executive summary first. "
        f"Improve clarity, keep bullet structure if present. Source:\n{research}"
    )
    if verbose:
        print(f"Output: {rewrite[:200]}...\n")
    
    return {
        'question': q,
        'research': research,
        'final': rewrite,
        'models': {
            'researcher': PRIMARY_MODEL_ID,
            'editor': EDITOR_MODEL_ID
        }
    }

# Execute sample pipeline
print("="*80)
result = pipeline('Explain why edge AI matters for compliance and latency.')
print("="*80)
print("\n[FINAL OUTPUT]")
print(result['final'])
print("\n[METADATA]")
print(f"Models used: {result['models']}")
result

[Pipeline] Question: Explain why edge AI matters for compliance and latency.

[Stage 1: Research]
Output: - **Data Sovereignty**: Edge AI allows data to be processed locally, which can help organizations comply with regional data protection regulations by keeping sensitive information within the borders o...

[Stage 2: Editorial Refinement]


### 解釋：管道執行及結果
執行多代理管道，針對一個以合規性和延遲為主題的問題進行演示：
- 多階段的信息轉化
- 代理的專業化及合作
- 通過精煉提升輸出質量
- 可追溯性（保留中間及最終輸出）

**結果結構：**
- `question` - 原始用戶查詢
- `research` - 原始研究輸出（事實性要點）
- `final` - 精煉的高層摘要
- `models` - 每個階段使用的模型

**擴展想法：**
1. 添加一個評論代理進行質量審查
2. 為不同方面實施並行研究代理
3. 添加一個驗證代理進行事實核查
4. 為不同複雜程度使用不同模型
5. 實施反饋循環以進行迭代改進


### 高級：自訂代理配置

嘗試在執行初始化單元格之前，通過修改環境變數來自訂代理行為：

**可用模型：**
- 在終端中使用 `foundry model ls` 查看所有可用模型
- 示例：phi-4-mini、phi-3.5-mini、qwen2.5-7b、llama-3.2-3b 等。


In [ ]:
# Example: Use different models for different agents
# Uncomment and modify as needed:

# import os
# os.environ['AGENT_MODEL_PRIMARY'] = 'phi-4-mini'      # Fast, good for research
# os.environ['AGENT_MODEL_EDITOR'] = 'qwen2.5-7b'       # Higher quality for editing

# Then restart the kernel and re-run all cells

# Test with different questions
test_questions = [
    "What are 3 key benefits of using small language models?",
    "How does RAG improve AI accuracy?",
    "Why is local inference important for privacy?"
]

print("Testing pipeline with multiple questions:\n")
for i, q in enumerate(test_questions, 1):
    print(f"\n{'='*80}")
    print(f"Question {i}: {q}")
    print('='*80)
    r = pipeline(q, verbose=False)
    print(f"\n[FINAL]: {r['final'][:300]}...")
    print(f"[Models]: Researcher={r['models']['researcher']}, Editor={r['models']['editor']}")


Testing pipeline with multiple questions:


Question 1: What are 3 key benefits of using small language models?

[FINAL]: <|channel|>analysis<|message|>The user wants a rewrite of the entire block of text. The rewrite should be professional, include a one-sentence executive summary first, improve clarity, keep bullet structure if present. The user has provided a large amount of text. The user wants a rewrite of that te...
[Models]: Researcher=Phi-4-mini-instruct-cuda-gpu:4, Editor=gpt-oss-20b-cuda-gpu:1

Question 2: How does RAG improve AI accuracy?

[FINAL]: <|channel|>final<|message|>**RAG (Retrieval‑Augmented Generation) empowers AI to produce highly accurate, contextually relevant responses by combining a retrieval system with a large language model (LLM).**<|return|>...
[Models]: Researcher=Phi-4-mini-instruct-cuda-gpu:4, Editor=gpt-oss-20b-cuda-gpu:1

Question 3: Why is local inference important for privacy?

[FINAL]: <|channel|>final<|message|>**Local inference—processing data d


---

**免責聲明**：  
本文件已使用 AI 翻譯服務 [Co-op Translator](https://github.com/Azure/co-op-translator) 進行翻譯。儘管我們致力於提供準確的翻譯，請注意自動翻譯可能包含錯誤或不準確之處。原始文件的母語版本應被視為權威來源。對於關鍵資訊，建議使用專業人工翻譯。我們對因使用此翻譯而引起的任何誤解或錯誤解釋概不負責。
